In [1]:
import os

from typing import Any, Mapping, List, Tuple, Dict

import numpy as np
import pandas as pd

from loguru import logger
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from pathlib import Path
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.nn.init import constant_, kaiming_normal_
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

from catalyst import dl, metrics
from catalyst.engines import Engine, CPUEngine, GPUEngine, DataParallelEngine

In [2]:
def get_available_engine() -> "Engine":
    if not torch.cuda.is_available():
        return CPUEngine()
    return GPUEngine() if torch.cuda.device_count() == 1 else DataParallelEngine()

In [3]:
class MultiVaeModel(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, latent_dim: int, dropout: float = 0.2) -> None:
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.dropout = dropout

        self.encoder_dims = [self.input_dim, self.hidden_dim, self.latent_dim * 2]
        self.decoder_dims = [self.latent_dim, self.hidden_dim, self.input_dim]

        self.encoder = self._build_layers(self.encoder_dims)
        self.decoder = self._build_layers(self.decoder_dims)

        self.apply(self._init_layer)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        h = nn.functional.dropout(x, self.dropout, training=self.training)
        h = self.encoder(h)

        mu = h[:, :self.latent_dim]
        log_var = h[:, self.latent_dim:]

        z = self._reparameterize(mu, log_var)
        z = self.decoder(z)
        return z, mu, log_var

    def reset(self) -> None:
        self.apply(self._init_layer)

    @staticmethod
    def _build_layers(dims: List[int]) -> nn.Sequential:
        layers = []
        for i in range(len(dims) - 1):
            layers.append(nn.Linear(dims[i], dims[i + 1]))
            if i + 1 < len(dims) - 1:
                layers.append(nn.BatchNorm1d(dims[i + 1]))
                layers.append(nn.ReLU())
        return nn.Sequential(*layers)

    @staticmethod
    def _init_layer(layer: nn.Module) -> None:
        if isinstance(layer, nn.Linear):
            kaiming_normal_(layer.weight.data)
            if layer.bias is not None:
                constant_(layer.bias.data, 0)

    def _reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

In [4]:
class MultiVaeDataset(Dataset):
    _embedding_ids: List[int]
    _embedding_id_to_embedding: Dict[int, np.ndarray[np.float64]]

    def __init__(self, embedding_ids: List[int], embedding_id_to_embedding: Dict[int, np.ndarray[np.float64]]) -> None:
        super(Dataset).__init__()
        self._embedding_ids = embedding_ids
        self._embedding_id_to_embedding = embedding_id_to_embedding

    def __len__(self) -> int:
        return len(self._embedding_ids)

    def __getitem__(self, index: int) -> Mapping[str, Any]:
        embedding_id = self._embedding_ids[index]
        embedding = self._embedding_id_to_embedding[embedding_id]
        return {"embedding_id": embedding_id, "embedding": embedding}

In [5]:
class MultiVaeRunner(dl.Runner):
    _loader_additive_metrics: Dict[str, metrics.AdditiveMetric]

    def __init__(self):
        super(MultiVaeRunner, self).__init__()

    @property
    def logger(self) -> Any:
        pass

    def on_loader_start(self, runner: dl.Runner) -> None:
        super().on_loader_start(runner)
        self._loader_additive_metrics = {
            metric_name: metrics.AdditiveMetric(compute_on_call=False)
            for metric_name in ["loss_ae", "loss_kld", "loss"]}

    def handle_batch(self, batch: Mapping[str, Any]) -> None:
        x = batch["embedding"]
        z, mu, log_var = self.model(x)

        anneal = min(self.hparams["anneal_cap"], self.batch_step / self.hparams["anneal_total_steps"])

        loss_ae = self._compute_loss_ae(x, z)
        loss_kld = self._compute_loss_kld(mu, log_var)
        loss = loss_ae + anneal * loss_kld

        self.batch_metrics = {"loss_ae": loss_ae, "loss_kld": loss_kld, "loss": loss}
        for metric_name, metric in self.batch_metrics.items():
            self._loader_additive_metrics[metric_name].update(metric.item(), self.batch_size)

    def on_loader_end(self, runner: dl.Runner) -> None:
        for metric_name, metric in self._loader_additive_metrics.items():
            self.loader_metrics[metric_name] = metric.compute()[0]
        super().on_loader_end(runner)

    def predict_batch(self, batch: Mapping[str, Any], **kwargs) -> Mapping[str, Any]:
        x = batch["embedding"]
        z, mu, log_var = self.model(x)
        return {**batch, "output": mu}

    @staticmethod
    def _compute_loss_ae(x: torch.Tensor, z: torch.Tensor) -> float:
        return -torch.mean(torch.sum(nn.functional.log_softmax(z, dim=1) * x, dim=1))

    @staticmethod
    def _compute_loss_kld(mu: torch.Tensor, log_var: torch.Tensor) -> float:
        return -0.5 * torch.mean(torch.sum(1 + log_var - mu.pow(2) - log_var.exp(), dim=1))

### Config

In [6]:
BASE_DIR = "/Users/artemvopilov/Programming/yandex_cup_2023"

In [37]:
NORMED_EMBEDDINGS_DIR = f"{BASE_DIR}/normed_embeddings"
VAE_EMBEDDINGS_DIR = f"{BASE_DIR}/vae_embeddings"

In [13]:
INPUT_DIM = 768
HIDDEN_DIM = 256
LATENT_DIM = 64

TRAIN_BATCH_SIZE = 256
VALID_BATCH_SIZE = 2048
INFERENCE_BATCH_SIZE = 8192

LR_SCHEDULER_STEP = 5

EPOCHS = 10

ANNEAL_CAP = 0.5
ANNEAL_TOTAL_STEPS = 20000

### Read embeddings

In [24]:
track_id_to_embeddings = {}
for fn in tqdm(os.listdir(NORMED_EMBEDDINGS_DIR)):
    fp = f"{NORMED_EMBEDDINGS_DIR}/{fn}"

    track_id = fn.split('.')[0]
    embeddings = np.load(fp)
    track_id_to_embeddings[track_id] = embeddings

  0%|          | 0/76714 [00:00<?, ?it/s]

### Prepare data

In [26]:
embedding_id_to_track_id_pos = {}
embedding_ids = []
embedding_id_to_embedding = {}
for ti, embeds in tqdm(track_id_to_embeddings.items()):
    for ei, embed in enumerate(embeds):
        embedding_id = len(embedding_id_to_track_id_pos)

        embedding_id_to_track_id_pos[embedding_id] = (ti, ei)
        embedding_ids.append(embedding_id)
        embedding_id_to_embedding[embedding_id] = embed

  0%|          | 0/76714 [00:00<?, ?it/s]

In [27]:
len(embedding_id_to_track_id_pos), len(embedding_ids), len(embedding_id_to_embedding)

(4452609, 4452609, 4452609)

### Train

In [12]:
train_embedding_ids, valid_embedding_ids = train_test_split(embedding_ids[:1000000], test_size=0.2)
logger.info(f"Divided df into train {len(train_embedding_ids)} and validation {len(valid_embedding_ids)}")

train_dataset = MultiVaeDataset(train_embedding_ids, embedding_id_to_embedding)
valid_dataset = MultiVaeDataset(valid_embedding_ids, embedding_id_to_embedding)
logger.info("Datasets created")

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE)
loaders = OrderedDict([('train', train_loader), ('valid', valid_loader)])
logger.info("Loaders created")

model = MultiVaeModel(input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM, dropout=0.1)
logger.info("Model initialized with config")
optimizer = Adam(params=model.parameters())
logger.info("Optimizer initialized")
lr_scheduler = StepLR(optimizer=optimizer, step_size=LR_SCHEDULER_STEP)
logger.info("Scheduler initialized")

num_epochs = EPOCHS

hparams = {"anneal_cap": ANNEAL_CAP, "anneal_total_steps": ANNEAL_TOTAL_STEPS}

engine = get_available_engine()
logger.info(f"Using engine {engine}")

callbacks = [
    dl.BackwardCallback(metric_key="loss"),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.SchedulerCallback(),
    dl.EarlyStoppingCallback(patience=2, loader_key="valid", metric_key="loss", minimize=True)]
logger.info(f"Callbacks created: {callbacks}")

runner = MultiVaeRunner()
runner.train(
    loaders=loaders,
    model=model,
    optimizer=optimizer,
    scheduler=lr_scheduler,
    num_epochs=num_epochs,
    hparams=hparams,
    engine=engine,
    verbose=True,
    timeit=False,
    callbacks=callbacks)

2023-11-06 17:31:01.575 | INFO     | __main__:<module>:2 - Divided df into train 800000 and validation 200000
2023-11-06 17:31:01.576 | INFO     | __main__:<module>:6 - Datasets created
2023-11-06 17:31:01.579 | INFO     | __main__:<module>:11 - Loaders created
2023-11-06 17:31:01.590 | INFO     | __main__:<module>:14 - Model initialized with config
2023-11-06 17:31:01.592 | INFO     | __main__:<module>:16 - Optimizer initialized
2023-11-06 17:31:01.595 | INFO     | __main__:<module>:18 - Scheduler initialized
2023-11-06 17:31:01.656 | INFO     | __main__:<module>:25 - Using engine <catalyst.engines.torch.CPUEngine object at 0x137242230>
2023-11-06 17:31:01.658 | INFO     | __main__:<module>:32 - Callbacks created: [<catalyst.callbacks.backward.BackwardCallback object at 0x137241690>, <catalyst.callbacks.optimizer.OptimizerCallback object at 0x13724a260>, <catalyst.callbacks.scheduler.SchedulerCallback object at 0x13724bd90>, <catalyst.callbacks.misc.EarlyStoppingCallback object at 0x1

1/10 * Epoch (train):   0%|          | 0/3125 [00:00<?, ?it/s]

train (1/10) loss: -50960.788538240995 | loss_ae: -50968.26594083464 | loss_kld: 126.1702626770017 | lr: 0.001 | momentum: 0.9


1/10 * Epoch (valid):   0%|          | 0/98 [00:00<?, ?it/s]

valid (1/10) loss: -143223.86907499997 | loss_ae: -143234.138545 | loss_kld: 64.70226213867187 | lr: 0.001 | momentum: 0.9
* Epoch (1/10) lr: 0.001 | momentum: 0.9


...

valid (9/10) loss: -2685532.2866399996 | loss_ae: -2685560.674 | loss_kld: 56.801065078125006 | lr: 0.0001 | momentum: 0.9
* Epoch (9/10) lr: 0.0001 | momentum: 0.9


10/10 * Epoch (train):   0%|          | 0/3125 [00:00<?, ?it/s]

train (10/10) loss: -2711708.5190400044 | loss_ae: -2711736.440799999 | loss_kld: 55.84546118652339 | lr: 0.0001 | momentum: 0.9


10/10 * Epoch (valid):   0%|          | 0/98 [00:00<?, ?it/s]

valid (10/10) loss: -2771299.6408799994 | loss_ae: -2771326.361839999 | loss_kld: 53.44989376220705 | lr: 0.0001 | momentum: 0.9
* Epoch (10/10) lr: 1e-05 | momentum: 0.9


### Inference

In [34]:
model.eval()

inference_dataset = MultiVaeDataset(embedding_ids, embedding_id_to_embedding)
inference_loader = DataLoader(inference_dataset, batch_size=INFERENCE_BATCH_SIZE)

batches_n = np.ceil(len(inference_dataset) / INFERENCE_BATCH_SIZE)
batches_5_perc = np.ceil(batches_n / 20)

logger.info(f"Computing embeddings by {batches_n} batches of size {INFERENCE_BATCH_SIZE}")

track_id_to_vae_embeddings = {ti: [None] * len(embeddings) for ti, embeddings in track_id_to_embeddings.items()}
batch_i = 0
for predictions in runner.predict_loader(loader=inference_loader, model=model, engine=engine):
    batch_embedding_ids = predictions["embedding_id"].detach().cpu().numpy()
    batch_vae_embeddings = predictions["output"].detach().cpu().numpy()

    for ei, vae_embed in zip(batch_embedding_ids, batch_vae_embeddings):
        ti, pos = embedding_id_to_track_id_pos[ei]
        track_id_to_vae_embeddings[ti][pos] = vae_embed

    batch_i += 1
    if batch_i % batches_5_perc == 0:
        logger.info(f'{round(100 * batch_i / batches_n, 2)} % batches processed')
logger.info("Computed embeddings")

2023-11-06 17:51:57.609 | INFO     | __main__:<module>:9 - Computing embeddings by 544.0 batches of size 8192
2023-11-06 17:52:03.770 | INFO     | __main__:<module>:23 - 5.15 % batches processed
2023-11-06 17:52:09.397 | INFO     | __main__:<module>:23 - 10.29 % batches processed
2023-11-06 17:52:14.687 | INFO     | __main__:<module>:23 - 15.44 % batches processed
2023-11-06 17:52:21.486 | INFO     | __main__:<module>:23 - 20.59 % batches processed
2023-11-06 17:52:26.967 | INFO     | __main__:<module>:23 - 25.74 % batches processed
2023-11-06 17:52:32.640 | INFO     | __main__:<module>:23 - 30.88 % batches processed
2023-11-06 17:52:38.920 | INFO     | __main__:<module>:23 - 36.03 % batches processed
2023-11-06 17:52:44.813 | INFO     | __main__:<module>:23 - 41.18 % batches processed
2023-11-06 17:52:51.020 | INFO     | __main__:<module>:23 - 46.32 % batches processed
2023-11-06 17:52:56.454 | INFO     | __main__:<module>:23 - 51.47 % batches processed
2023-11-06 17:53:02.187 | INFO 

In [35]:
len(track_id_to_vae_embeddings)

76714

### Save

In [40]:
os.mkdir(VAE_EMBEDDINGS_DIR)

In [41]:
for ti, embeddings in tqdm(track_id_to_vae_embeddings.items()):
    fn = f"{ti}.npy"
    fp = f"{VAE_EMBEDDINGS_DIR}/{fn}"
    np.save(fp, embeddings)

  0%|          | 0/76714 [00:00<?, ?it/s]